In [1]:
import pandas
from konlpy.tag import Komoran
from gensim.models import word2vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import sys

In [2]:
#파일 읽어오기
file = open('./test.txt','r',encoding='utf-8')
line = file.read()
lines = line[:19*350]

#장(chapter)마다 나눔
sentences = re.split('(?<=[2-9]\n)|(?<=[0-9][0-9]\n)|(?<=[0-9][0-9][0-9]\n)', lines)
# if len(sentences) < 2:
#     sentences = re.split('(?<=[0-9]장)|(?<=[0-9][0-9]장)', lines)

#Komoran 형태소 분석을 위한 전처리
sent=[]
for stuff in sentences:
        stuff= "\n".join([s for s in stuff.split("\n") if s])
        stuff="".join([s for s in stuff.splitlines(True) if s.strip()])
        sent.append(stuff)

In [3]:
print(sent[0])

호질
  범은 착하고도 효성스러우며, 문채롭고도 싸움을 잘한다. 인자하고도 효성스러우며, 슬기롭고도 어질다. 씩씩하고도 날래며, 세차고도 사납다. 그야말로 천하에 대적할 자가 없다.
그러나 비위는 범을 잡아먹고, 범우도 범을 잡아 먹는다. 박(駁)도 범을 잡아먹고, 오색사자는 큰 나무가 선 산꼭대기에서 범을 잡아먹는다. 자백도 범을 잡아먹고, 표견은 날면서 범과 표범을 잡아먹는다. 황요는 범과 표범의 염통을 꺼내어 먹는다. 활(猾)은 범과 표범에게 일부러 삼켜졌다가 그 뱃속에서 간을 뜯어 먹고, 추이(酋耳)는 범을 만나기만 하며 곧 찢어서 먹는다. 범이 맹용을 만나면 눈을 꼭 감고, 감히 뜨지도 못한다. 그런데 사람이 맹용을 두려워하지 않으면서도 범은 두려워 하니, 범의 위풍이 얼마나 엄한가.
범이 개를 먹으면 취하고, 사람을 먹으면 조화를 부리게 된다. 범이 한 번 사람을 먹으면, 그 창귀가 굴각(屈閣)이 되어 범의 겨드랑이에 붙어산다. 굴각이 범을 남의 집 부엌으로 이끌어 들여서 솥전을 핥으면, 그 집주인이 갑자기 배고픈 생각이 나서 한밤중이라도 아내더러 밥을 지으라고 시키게 된다. 범이 두 번째로 사람을 먹으련, 그 창귀가 이올(彛兀)이 되어 범의 광대뼈에 붙어산다. 이올은 높은 데 올라가서 사냥꾼의 움직임을 살피는데, 만약 깊은 골짜기에 함정이나 묻힌 화살이 있으면 먼저 가서 그 틀을 벗겨 놓는다. 범이 세번째로 사람을 먹으면, 그 창귀가 육혼(　渾)이 되어 범의 턱에 붙어산다. 육혼은 자기가 평소에 알던 친구들의 이름을 자꾸만 불러댄다.
하루는 범이 창귀들에게 분부를 내렸다. 
"오늘도 해가 저무니, 어디서 먹을 것을 얻을까?"
굴각은 이렇게 말하였다.
"제가 아까 점을 쳐 보았더니 뿔 있는 놈도 아니고 날짐승도 아닌, 검은머리를 한 놈이 나왔습니다. 눈 위에 발자국이 있는데, 비틀비틀 성긴 걸음이었습니다. 뒤통수에 꼬리가 붙고, 꽁무니를 감추지 못하는 놈이었습니다."
이올은 이렇게 말하였다.
"동문(東門)에 먹을 것이 있는데, 이름은 의원(醫員)이라

In [4]:
twitter = Komoran(userdic='./user_dic.txt')

In [5]:
import timeit
start = timeit.default_timer()

#형태소분석
sentences_tag = []
for sentence in sent:
    morph = twitter.pos(sentence)
    sentences_tag.append(morph)
    
stop = timeit.default_timer()
print(stop - start)

0.43843679999986307


In [6]:
#명사 추출
noun_adj_list=[]
noun_adj=[]
#komoran
for sentence1 in sentences_tag:
    noun_adj=[]
    for word, tag in sentence1:
        if word in '왕' or len(word)>1:
            if tag in ['NNP']: #komoran
                word = word.replace(" ","")
                noun_adj.append(word)
    noun_adj_list.append(noun_adj)

In [7]:
print(noun_adj)
print(noun_adj_list)

['호질', '우도', '사자', '산꼭대기', '자백', '표범', '표범', '표범', '위풍', '겨드랑이', '한밤', '광대뼈', '골짜기', '함정', '화살', '세번', '오늘', '제가', '머리', '발자국', '비틀', '비틀', '꼬리', '동문', '서문', '목욕', '수염', '노여움', '쓸개', '지조', '이고', '백가', '석덕', '지유', '오미', '눈썹', '오행', '육기', '말투', '음양', '오행', '육기', '남이', '구역질', '북곽', '손수', '구경', '오천', '천자', '제후', '청춘', '과부', '동리자', '라고', '천자', '제후', '어진', '사방', '동리', '과부', '동리자', '나니', '북곽', '선생', '동리자', '북곽', '선생', '오늘밤', '선생님', '북곽', '선생', '병풍', '원앙', '가마솥', '무어', '예기', '과부', '북곽', '어진', '성문', '여우', '여우', '북곽', '여우', '여우', '여우', '아이들', '북곽', '도깨비', '도깨비', '구덩이', '구역질', '이쿠', '북곽', '선생', '엉금엉금', '기어', '대인', '왕', '효성', '사모', '장수', '신룡', '천한', '앞으로', '천하', '천하', '성품', '성품', '성품', '성품', '오상', '사강', '서울', '베이', '그럼에도불구하고', '밧줄', '바늘', '도끼', '형벌', '형벌', '성품', '벌레', '노루', '송사', '광명', '노루', '원수', '노루', '사슴', '소가', '일해', '갈기', '우리들의', '노루', '사슴', '장수', '메뚜기', '누에', '개미', '하늘', '메뚜기', '누에', '개미', '개미', '천지', '거도', '메뚜기', '누에', '대적', '표범', '노루', '사슴', '노루', '사슴', '관중', '산동', '춘추', '시대', '춘추', '시대', 

In [8]:
############# TF-IDF###################

In [9]:
# tf-idf 전처리
ad_text=[]
b=''
for i in noun_adj_list:
    a=" ".join(i).split(",")
    print(a)
    b=''.join(a)
    print(b)
    ad_text.append(b)

['호질 우도 사자 산꼭대기 자백 표범 표범 표범 위풍 겨드랑이 한밤 광대뼈 골짜기 함정 화살 세번 오늘 제가 머리 발자국 비틀 비틀 꼬리 동문 서문 목욕 수염 노여움 쓸개 지조 이고 백가 석덕 지유 오미 눈썹 오행 육기 말투 음양 오행 육기 남이 구역질 북곽 손수 구경 오천 천자 제후 청춘 과부 동리자 라고 천자 제후 어진 사방 동리 과부 동리자 나니 북곽 선생 동리자 북곽 선생 오늘밤 선생님 북곽 선생 병풍 원앙 가마솥 무어 예기 과부 북곽 어진 성문 여우 여우 북곽 여우 여우 여우 아이들 북곽 도깨비 도깨비 구덩이 구역질 이쿠 북곽 선생 엉금엉금 기어 대인 왕 효성 사모 장수 신룡 천한 앞으로 천하 천하 성품 성품 성품 성품 오상 사강 서울 베이 그럼에도불구하고 밧줄 바늘 도끼 형벌 형벌 성품 벌레 노루 송사 광명 노루 원수 노루 사슴 소가 일해 갈기 우리들의 노루 사슴 장수 메뚜기 누에 개미 하늘 메뚜기 누에 개미 개미 천지 거도 메뚜기 누에 대적 표범 노루 사슴 노루 사슴 관중 산동 춘추 시대 춘추 시대 시체 가뭄 원수 무당 세속 병장 기도 까마귀 솔개 개미 대궁 이루 이루 함정 그물 노루 그물 물고기 그물 물고기 그물 수레 그물 삼태 그물 그물 바늘 도끼 화포 화산 아교 치도 오징어']
호질 우도 사자 산꼭대기 자백 표범 표범 표범 위풍 겨드랑이 한밤 광대뼈 골짜기 함정 화살 세번 오늘 제가 머리 발자국 비틀 비틀 꼬리 동문 서문 목욕 수염 노여움 쓸개 지조 이고 백가 석덕 지유 오미 눈썹 오행 육기 말투 음양 오행 육기 남이 구역질 북곽 손수 구경 오천 천자 제후 청춘 과부 동리자 라고 천자 제후 어진 사방 동리 과부 동리자 나니 북곽 선생 동리자 북곽 선생 오늘밤 선생님 북곽 선생 병풍 원앙 가마솥 무어 예기 과부 북곽 어진 성문 여우 여우 북곽 여우 여우 여우 아이들 북곽 도깨비 도깨비 구덩이 구역질 이쿠 북곽 선생 엉금엉금 기어 대인 왕 효성 사모 장수 신룡 천한 앞으로 천하 천하 성품 성품 성품 성품 오상 사강 서울 베이 그럼에도불구하고 밧

In [10]:
print(ad_text)

['호질 우도 사자 산꼭대기 자백 표범 표범 표범 위풍 겨드랑이 한밤 광대뼈 골짜기 함정 화살 세번 오늘 제가 머리 발자국 비틀 비틀 꼬리 동문 서문 목욕 수염 노여움 쓸개 지조 이고 백가 석덕 지유 오미 눈썹 오행 육기 말투 음양 오행 육기 남이 구역질 북곽 손수 구경 오천 천자 제후 청춘 과부 동리자 라고 천자 제후 어진 사방 동리 과부 동리자 나니 북곽 선생 동리자 북곽 선생 오늘밤 선생님 북곽 선생 병풍 원앙 가마솥 무어 예기 과부 북곽 어진 성문 여우 여우 북곽 여우 여우 여우 아이들 북곽 도깨비 도깨비 구덩이 구역질 이쿠 북곽 선생 엉금엉금 기어 대인 왕 효성 사모 장수 신룡 천한 앞으로 천하 천하 성품 성품 성품 성품 오상 사강 서울 베이 그럼에도불구하고 밧줄 바늘 도끼 형벌 형벌 성품 벌레 노루 송사 광명 노루 원수 노루 사슴 소가 일해 갈기 우리들의 노루 사슴 장수 메뚜기 누에 개미 하늘 메뚜기 누에 개미 개미 천지 거도 메뚜기 누에 대적 표범 노루 사슴 노루 사슴 관중 산동 춘추 시대 춘추 시대 시체 가뭄 원수 무당 세속 병장 기도 까마귀 솔개 개미 대궁 이루 이루 함정 그물 노루 그물 물고기 그물 물고기 그물 수레 그물 삼태 그물 그물 바늘 도끼 화포 화산 아교 치도 오징어']


In [11]:
if len(sentences) < 2 :
    ad_text = noun_adj

In [12]:
print(ad_text)

['호질', '우도', '사자', '산꼭대기', '자백', '표범', '표범', '표범', '위풍', '겨드랑이', '한밤', '광대뼈', '골짜기', '함정', '화살', '세번', '오늘', '제가', '머리', '발자국', '비틀', '비틀', '꼬리', '동문', '서문', '목욕', '수염', '노여움', '쓸개', '지조', '이고', '백가', '석덕', '지유', '오미', '눈썹', '오행', '육기', '말투', '음양', '오행', '육기', '남이', '구역질', '북곽', '손수', '구경', '오천', '천자', '제후', '청춘', '과부', '동리자', '라고', '천자', '제후', '어진', '사방', '동리', '과부', '동리자', '나니', '북곽', '선생', '동리자', '북곽', '선생', '오늘밤', '선생님', '북곽', '선생', '병풍', '원앙', '가마솥', '무어', '예기', '과부', '북곽', '어진', '성문', '여우', '여우', '북곽', '여우', '여우', '여우', '아이들', '북곽', '도깨비', '도깨비', '구덩이', '구역질', '이쿠', '북곽', '선생', '엉금엉금', '기어', '대인', '왕', '효성', '사모', '장수', '신룡', '천한', '앞으로', '천하', '천하', '성품', '성품', '성품', '성품', '오상', '사강', '서울', '베이', '그럼에도불구하고', '밧줄', '바늘', '도끼', '형벌', '형벌', '성품', '벌레', '노루', '송사', '광명', '노루', '원수', '노루', '사슴', '소가', '일해', '갈기', '우리들의', '노루', '사슴', '장수', '메뚜기', '누에', '개미', '하늘', '메뚜기', '누에', '개미', '개미', '천지', '거도', '메뚜기', '누에', '대적', '표범', '노루', '사슴', '노루', '사슴', '관중', '산동', '춘추', '시대', '춘추', '시대', 

In [13]:
#TF
# vector = CountVectorizer()
# print(vector.fit_transform(ad_text).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
# print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

In [14]:
# TF-IDF 설정
tfidf = TfidfVectorizer(max_features = 5, max_df=0.95, min_df=0) #(max_features = 5) <- 상위 5개만 추출

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(ad_text)

In [15]:
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict) 

['그물', '노루', '북곽', '성품', '여우']


In [16]:
# 값이 적을수록 다른 문서에서 자주 언급
print(tfidf.vocabulary_)

{'북곽': 2, '여우': 4, '성품': 3, '노루': 1, '그물': 0}


In [17]:
# counter 위주
# wh=[]
# for i in vector.vocabulary_:
#     if i in tfidf_dict:
#         wh.append(i)
#         print(i)
# print(wh)

# import collections 
# container = collections.Counter()
# container.update(noun_adj_all)

# for k,v in container.items():
#     if k in tfidf_dict:
#         print(k,":",v)

In [18]:
#############Word2Vec######################

In [19]:
sents = [doc.split(" ") for doc in ad_text] #Word2Vec 을 위한 전처리

In [20]:
print(sents)

[['호질'], ['우도'], ['사자'], ['산꼭대기'], ['자백'], ['표범'], ['표범'], ['표범'], ['위풍'], ['겨드랑이'], ['한밤'], ['광대뼈'], ['골짜기'], ['함정'], ['화살'], ['세번'], ['오늘'], ['제가'], ['머리'], ['발자국'], ['비틀'], ['비틀'], ['꼬리'], ['동문'], ['서문'], ['목욕'], ['수염'], ['노여움'], ['쓸개'], ['지조'], ['이고'], ['백가'], ['석덕'], ['지유'], ['오미'], ['눈썹'], ['오행'], ['육기'], ['말투'], ['음양'], ['오행'], ['육기'], ['남이'], ['구역질'], ['북곽'], ['손수'], ['구경'], ['오천'], ['천자'], ['제후'], ['청춘'], ['과부'], ['동리자'], ['라고'], ['천자'], ['제후'], ['어진'], ['사방'], ['동리'], ['과부'], ['동리자'], ['나니'], ['북곽'], ['선생'], ['동리자'], ['북곽'], ['선생'], ['오늘밤'], ['선생님'], ['북곽'], ['선생'], ['병풍'], ['원앙'], ['가마솥'], ['무어'], ['예기'], ['과부'], ['북곽'], ['어진'], ['성문'], ['여우'], ['여우'], ['북곽'], ['여우'], ['여우'], ['여우'], ['아이들'], ['북곽'], ['도깨비'], ['도깨비'], ['구덩이'], ['구역질'], ['이쿠'], ['북곽'], ['선생'], ['엉금엉금'], ['기어'], ['대인'], ['왕'], ['효성'], ['사모'], ['장수'], ['신룡'], ['천한'], ['앞으로'], ['천하'], ['천하'], ['성품'], ['성품'], ['성품'], ['성품'], ['오상'], ['사강'], ['서울'], ['베이'], ['그럼에도불구하고'], ['밧줄'], ['바늘'], ['도끼'], ['형벌'], ['형벌'], ['성

In [21]:
model = word2vec.Word2Vec(sents, size=200,window=100,hs=1,min_count=1,sg=1)

print(list(model.wv.vocab.keys()))
print("vocab length : %d"%len(model.wv.vocab))


['호질', '우도', '사자', '산꼭대기', '자백', '표범', '위풍', '겨드랑이', '한밤', '광대뼈', '골짜기', '함정', '화살', '세번', '오늘', '제가', '머리', '발자국', '비틀', '꼬리', '동문', '서문', '목욕', '수염', '노여움', '쓸개', '지조', '이고', '백가', '석덕', '지유', '오미', '눈썹', '오행', '육기', '말투', '음양', '남이', '구역질', '북곽', '손수', '구경', '오천', '천자', '제후', '청춘', '과부', '동리자', '라고', '어진', '사방', '동리', '나니', '선생', '오늘밤', '선생님', '병풍', '원앙', '가마솥', '무어', '예기', '성문', '여우', '아이들', '도깨비', '구덩이', '이쿠', '엉금엉금', '기어', '대인', '왕', '효성', '사모', '장수', '신룡', '천한', '앞으로', '천하', '성품', '오상', '사강', '서울', '베이', '그럼에도불구하고', '밧줄', '바늘', '도끼', '형벌', '벌레', '노루', '송사', '광명', '원수', '사슴', '소가', '일해', '갈기', '우리들의', '메뚜기', '누에', '개미', '하늘', '천지', '거도', '대적', '관중', '산동', '춘추', '시대', '시체', '가뭄', '무당', '세속', '병장', '기도', '까마귀', '솔개', '대궁', '이루', '그물', '물고기', '수레', '삼태', '화포', '화산', '아교', '치도', '오징어']
vocab length : 128


In [22]:
wv = model.wv #Word2Vec model
# del(model)
# 전달받은 sigwords 중 model에 포함된 단어를 우선순위 순서로 추려냄
sigvocs = []
missvocs = []
hit = 0
miss = 0
for worddic in tfidf_dict:
    try:
        word = worddic
        if wv.vocab[word]:
            sigvocs.append(word)
            hit += 1
        if hit >= length:
            break
    except:
        missvocs.append(word)
        miss += 1
# 디버깅용. 추후 삭제.
print("missed: %d"  % miss)
for voc in missvocs:
    print(voc)


# 디스턴스 매트릭스를 데이터프레임 형태로 만듦
distDF =  pandas.DataFrame(columns = sigvocs , index = sigvocs)
for i in sigvocs:
    dists = []
    for j in sigvocs:
        dist = round(wv.distance(i, j), 6)
        dists.append(0 if dist < 1.0e-2 else dist)
    distDF[i] = dists
print(distDF)

missed: 5
그물
노루
북곽
성품
여우
          그물        노루        북곽        성품        여우
그물  0.000000  0.988213  1.110388  1.143529  1.131892
노루  0.988213  0.000000  0.902418  1.020644  1.048807
북곽  1.110388  0.902418  0.000000  0.937863  0.875537
성품  1.143529  1.020644  0.937863  0.000000  1.011816
여우  1.131892  1.048807  0.875537  1.011816  0.000000


In [23]:
for i in sigvocs:
    print(i)
    print(wv.most_similar(positive=i, negative=[]))

그물
[('손수', 0.16520382463932037), ('청춘', 0.1404542326927185), ('수염', 0.12327784299850464), ('노여움', 0.12124629318714142), ('동리', 0.11700242757797241), ('동문', 0.11124545335769653), ('도깨비', 0.10494834184646606), ('어진', 0.09458916634321213), ('기도', 0.09237266331911087), ('갈기', 0.08223442733287811)]
노루
[('시대', 0.17966288328170776), ('함정', 0.1682264804840088), ('자백', 0.16180522739887238), ('세속', 0.15738806128501892), ('아이들', 0.1425621062517166), ('사슴', 0.14237435162067413), ('나니', 0.12443433701992035), ('누에', 0.11980461329221725), ('겨드랑이', 0.11623746901750565), ('베이', 0.10774224996566772)]
북곽
[('한밤', 0.15483692288398743), ('말투', 0.15414327383041382), ('일해', 0.14782468974590302), ('골짜기', 0.14480099081993103), ('대적', 0.1439664363861084), ('어진', 0.14153897762298584), ('그럼에도불구하고', 0.1393870711326599), ('선생님', 0.1363590955734253), ('원수', 0.1332012116909027), ('세속', 0.1245412677526474)]
성품
[('오행', 0.20140254497528076), ('아이들', 0.1986590027809143), ('형벌', 0.16178224980831146), ('앞으로', 0.125171363353

In [24]:
search=[]
for i in sigvocs:
    for j in distDF[i].sort_values().index[1:3]:
           search.append([i,j])

In [25]:
for i in search:
#     i = str(i)
#     i = i.replace("(","").replace(")","").replace("'","").replace(" ","").split(",")
    print((i))
    print(wv.most_similar(positive=i, negative=[]))

['그물', '노루']
[('손수', 0.17574380338191986), ('함정', 0.16618612408638), ('세속', 0.14990836381912231), ('청춘', 0.13103854656219482), ('시대', 0.12225139141082764), ('도깨비', 0.10889527201652527), ('누에', 0.10441048443317413), ('노여움', 0.10144162178039551), ('자백', 0.10141514986753464), ('예기', 0.09924833476543427)]
['그물', '북곽']
[('어진', 0.17702409625053406), ('손수', 0.16222020983695984), ('한밤', 0.13986654579639435), ('원수', 0.13836050033569336), ('골짜기', 0.1367325782775879), ('세속', 0.13524600863456726), ('청춘', 0.1310664564371109), ('대적', 0.12213774025440216), ('그럼에도불구하고', 0.12195643782615662), ('선생', 0.11602199077606201)]
['노루', '북곽']
[('세속', 0.19028596580028534), ('자백', 0.18170566856861115), ('선생님', 0.1616595834493637), ('시대', 0.15775123238563538), ('그럼에도불구하고', 0.1422417014837265), ('베이', 0.13950027525424957), ('원수', 0.1329728364944458), ('대적', 0.12672623991966248), ('제후', 0.11443810909986496), ('밧줄', 0.11318975687026978)]
['노루', '그물']
[('손수', 0.17574380338191986), ('함정', 0.16618612408638), ('세속', 0.14

In [26]:
############################################

In [24]:
################################

In [33]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
np.random.seed(0)

from konlpy.tag import Okt
twitter = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import HashingVectorizer

In [34]:
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [35]:

file = open('./TheLittlePrince.txt','r',encoding='utf-8')
raw_lines = file.readlines()

# 2. 변수 rawdata 저장
rawdata = []
for line in raw_lines:
#     line= "\n".join([s for s in stuff.split("\n") if s])
    rawdata.append(line)

In [36]:
print(rawdata)

['\ufeff여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨 그린 것이다.\n', ' 그 책에는 이렇게 씌어 있었다.\n', ' "보아 구렁이는 먹이를 씹지도 않고 통째로 집어삼킨다.그리고는 꼼짝도 하지 못하고 여섯 달 동안 잠을 자면서 그것을 소화시킨다."\n', ' 나는 그래서 밀림 속에서의 모험에 대해 한참 생각해 보고 난 끝에 색연필을 가지고 내 나름대로 내 생애 첫번째 그림을 그려보았다. 나의 그림 제 1호였다. 그것은 이런 그림이었다.\n', ' 나는 그 걸작품을 어른들에게 보여 주면서내 그림이 무섭지 않느냐고 물었다.\n', ' 그들은 "모자가 뭐가 무섭다는 거니?" 하고 대답했다.\n', ' 내 그림은 모자를 그린 게 아니었다. 그것은 코끼리를 소화시키고 있는 보아 구렁이었다.\n', ' 그래서 나는 어른들이 알아볼 수 있도록 보아 구렁이의 속을 그렸다. 어른들은 언제나 설명을 해주어야만 한다. 나의 그림 제 2호는 이러했다.\n', ' 어른들은 속이 보이거나 보이지 않거나 하는 보아 구렁이의 그림들은 집어치우고 차라리 지리, 역사, 계산, 그리고 문법 쪽에 관심을 가져보는 게 좋을 것이라고 충고해 주었다.\n', ' 그래서 나는 여섯 살 적에 화가라는 멋진 직업을 포기해 버렸다.내 그림제 1호와 제 2호가 성공을 거두지 못한 데 낙심해 버렸던 것이다. 어른들은언제나 스스로는 아무것도 이해하지 못한다.자꾸자꾸 설명을 해주어야 하니 맥빠지는 노릇이 아닐 수 없다.\n', ' 그래서 다른 직업을 선택하지 않을 수 없게 된 나는 비행기 조종하는 법을배웠다.세계의 여기저기 거의 안 가본 데 없이 나는 날아다녔다.그러니지리는 정말로 많은 도움을 준 셈이었다.한번 슬쩍 보고도 중국과 애리조나를 나는 구별할 수 있었던 것이다.그것은 밤에 길을 잃었을 때 아주 유용한 일이다.\n', ' 나는 그리하여 일생 동안 수없이 많은 점잖

In [37]:
vectorize = HashingVectorizer(
    tokenizer=tokenizer,
    n_features=7               # 기본 feature 수를 설정하며 기본값이 2의 20승이다. 아래 예시를 위해 feature 를 7로 한정했으나, 아래 유사문장을 찾을때는 다시 n_features 주석처리 했다.
)
X = vectorize.fit_transform(rawdata)
 
print(X.shape)
 
print(X.toarray())

(977, 7)
[[-0.26726124 -0.26726124  0.26726124 ...  0.         -0.26726124
   0.80178373]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.37796447 -0.75592895  0.         ...  0.          0.
   0.37796447]
 ...
 [ 0.51639778  0.51639778  0.         ... -0.25819889 -0.25819889
  -0.51639778]
 [-0.12216944  0.85518611  0.24433889 ...  0.24433889 -0.24433889
   0.12216944]
 [ 0.          0.52075564  0.13018891 ... -0.52075564 -0.13018891
  -0.39056673]]


In [42]:

# search 문장 벡터
srch_vector = vectorize.transform([
    '화산'
])
 
print(srch_vector.shape)
# (1, 7)


(1, 7)


In [43]:
from sklearn.metrics.pairwise import linear_kernel
 
# linear_kernel는 두 벡터의 dot product 이다.
cosine_similar = linear_kernel(srch_vector, X).flatten()
# cosine_similar = (srch_vector*X.T).toarray().flatten()
 
print(cosine_similar)
# [0.         0.         0.62017367 0.31622777 0.3]
 
print(cosine_similar.shape)
# (5,)
 

[ 0.          0.          0.          0.          0.          0.
  0.5         0.         -0.3592106   0.12309149  0.         -0.28867513
 -0.51604685 -0.40824829  0.          0.21821789  0.          0.
  0.          0.          0.          0.          0.          0.
 -0.28867513 -0.57735027 -0.75592895  0.          0.          0.16439899
  0.70710678  0.5         0.          0.         -0.5        -0.57735027
  0.          0.          0.         -0.70710678  0.31622777  0.
  0.          1.          1.         -0.40824829  0.          0.
  0.         -0.47140452  0.          0.          0.         -0.57735027
  0.          0.          0.          0.63245553  0.          0.
  0.          0.          0.          0.          0.30151134 -0.57735027
  0.         -0.81649658 -0.5         0.66666667  0.57735027  0.40824829
 -0.57735027  0.          0.          0.          1.          0.5
  0.          0.         -0.64051262  0.          0.35355339  0.2236068
  0.1796053   0.53452248 -0.377964

In [44]:

# 유사한 rawdata index
sim_rank_idx = cosine_similar.argsort()[::-1]
print(sim_rank_idx)
#[2 3 4 1 0]
 

[175  76 472 841  44 112  43 567 152 972 502 201 890 189 181 482 366 718
 362 360 908 352 270 549 484 504 512  30 507 501 481 818  69 643 897  57
 287 211 345 340 364 132 658 765 711 485 476 478  70  85 125 247 174 871
 518 560   6  77 475 937  31 907 804 180 384 615 350 193 561 795 576 219
 115 499 812 535  71 923 130 666 250 225  89 806 305 809 862 346 623 403
 798 903 426 408 613 465 116 261 514 110  82 892 449 726 592 406 101 854
 284  40 859  64 208 675 432 786 232 111 975 195  83  15 229 716 962 170
 819  84  29 970 265 900   9 490 240 489 494 113 493 483 487 216 215 114
 239 470 480 479 119 120 122 456 123 454 453 124 251 450 252 448 126 445
 444 118 461 462 212 243 244 477 245 246 213 471 463 238 469 468 467 466
 464 495 233 497 546 552 551 550 227  97 547 545 538 100 543 542 541 540
 218 553 554 555  95 557 559 562 563 564 566 568 569  88 571 224 573 574
 539 537 498 509 513 217 231 109 510 442 508 536 506 234 235 236 237 500
 515 107 517 519 106 105 522 523 524 525 526 527 10

In [45]:
for i in sim_rank_idx:
    if cosine_similar[i] > 0:
        print('{} / score : {}'.format(rawdata[i], cosine_similar[i]))

 "참 아름다우시군요!"
 / score : 1.0
 "괜찮아. 내가 사는 곳은 아주 작으니까!"
 / score : 1.0
 "지리학자가 뭐에요?"
 / score : 1.0
 그러자 나는 그가 무엇을 찾고 있었는지를 깨달았다.
 / score : 1.0
 "거기 있는 걸로 아마 충분할 거다. 네게 준 건 아주 작은 양이니까."
 / score : 1.0
 "그건 기율(紀律)의 문제야."
 / score : 1.0
 "내가 사는 곳은 아주 작거든......"
 / score : 1.0
 "난 배보다 더 먼 곳으로 너를 데려다줄 수 있어."
 / score : 1.0
 그는 깜짝 놀라서 나를 바라보았다.
 / score : 0.9486832980505138
 어느 때는 (한두 번 방심할 수도 있지. 그러면 끝장인데!어느날 밤 그가유리덮개를 잊었거나 양이 밤중에 소리없이 밖으로 나왔을지도 몰라......)하는 생각이 들기도 한다. 그러면 작은 방울들은 모두 눈물방울로 변한다!......
 / score : 0.944911182523068
 "아, 내 별은 별로 흥미로울 게 없어요. 아주 작거든요. 화산이 셋있어요. 둘은 불이 있는 화산이고 하나는 불이 꺼진 화산이지요. 하지만 언제 어떻 게 될지 모르지요."
 / score : 0.8320502943378437
 나는 어린 왕자가 철새들의 이동을 이용하여 별을 떠나왔으리라 생각한다.떠나는날 아침 그는 그의 별을 잘 정돈해 놓았다. 불을 뿜는 화산들은 정성스레 쑤셔서 청소했다. 그에게는 불을 뿜는 화산이 둘 있었다. 그런데 그것은 아침밥을 데우는 데 아주 편리했다. 불이 꺼져 있는 화산도 하나 있었다그러나 그의 말처럼"어떻게 될지 알 수 없는 일이었다."그는 그래서 불꺼진 화산도 잘 쑤셔 놓았다.화산들은 잘 청소되어있을 때는 부드럽게,규칙적으로, 폭발하지 않고 타오른다. 화산의 폭발은 벽난로의 불과 마찬가지인 것이다. 물론 우리 지구 위에서는 우리들의 화산을 쑤시기에는 우리가너무 작다. 그래서 화산이 우리에게 숱한